# Identify key datasets

In [1]:
!pip install geopandas

In [2]:
from calitp.tables import tbl
from calitp import query_sql
from siuba import *

import geopandas as gpd
import pandas as pd
import os

pd.set_option("display.max_rows", 20)
#os.environ["CALITP_BQ_MAX_BYTES"] = 20_000_000_000

In [3]:
# CalEnviroScreen
CALENVIRONSCREEN = './calenviroscreen40shp_F_2021/CES4_final.shp'

gdf = gpd.read_file(CALENVIRONSCREEN)
gdf.head()

,Tract,ZIP,Population,CIscore,CIscoreP,Ozone,Ozone_Pctl,PM2_5,PM2_5_Pctl,Diesel_PM,...,African_Am,Native_Ame,Asian_Amer,Pacific_Is,Other_Mult,Shape_Leng,Shape_Area,County,City_1,geometry
0,6.083002e+09,93454,4495,36.019653,69.162885,0.034190,10.566273,7.567724,10.031114,0.154573,...,0.4004,0.2670,8.2091,0.0,1.3126,6999.357622,2.847611e+06,Santa Barbara,Santa Maria,"POLYGON ((-39795.070 -341919.191, -38126.384 -..."
1,6.083002e+09,93455,13173,37.030667,70.637922,0.035217,11.561917,7.624775,10.454263,0.106088,...,2.5051,0.0000,4.6990,0.0,0.9489,19100.578003,1.635292e+07,Santa Barbara,Santa Maria,"POLYGON ((-39795.070 -341919.191, -39803.632 -..."
2,6.083002e+09,93454,2398,31.213140,61.069087,0.034190,10.566273,7.548835,9.931549,0.190474,...,0.9591,0.0000,0.5421,0.0,2.1685,4970.985933,1.352330e+06,Santa Barbara,Santa Maria,"POLYGON ((-38115.747 -341130.248, -38126.384 -..."
3,6.083002e+09,93455,4496,6.639331,5.988401,0.036244,13.615432,7.660570,10.653391,0.091384,...,0.9342,0.7117,3.6699,0.0,2.5356,6558.956142,2.417717e+06,Santa Barbara,Orcutt,"POLYGON ((-37341.662 -348530.437, -37252.307 -..."
4,6.083002e+09,93455,4008,14.022852,23.121533,0.036244,13.615432,7.663210,10.690728,0.050976,...,0.6986,1.4721,3.2685,0.0,1.3723,6570.368642,2.608422e+06,Santa Barbara,Orcutt,"POLYGON ((-39465.107 -348499.262, -38244.305 -..."


In [4]:
def define_equity_groups(df, percentile_col = ["CIscoreP"], num_groups=5):
    for col in percentile_col:
        df = df.assign(group_col = 0)

        bin_range = round(100 / num_groups)

        for i in range(1, num_groups + 1):
            max_cutoff = i * bin_range
            df = df.assign(
                group_col = df.apply(
                    lambda x: i if (x[col] <= max_cutoff) and 
                    (x[col] >= max_cutoff - 19)
                    else x.group_col, axis = 1),
            )
        df = df.rename(columns = {"group_col": f"{col}_group"})
    
    return df

In [5]:
gdf = define_equity_groups(gdf, 
                           percentile_col =  ["CIscoreP", "Pollution_", "PopCharP"], 
                           num_groups=5)

In [6]:
# Double check
gdf[["CIscoreP", "CIscoreP_group", 
     "Pollution_", "Pollution__group", 
     "PopCharP", "PopCharP_group"]].head()

,CIscoreP,CIscoreP_group,Pollution_,Pollution__group,PopCharP,PopCharP_group
0,69.162885,4,45.003111,3,79.828543,4
1,70.637922,4,79.962663,4,56.555724,3
2,61.069087,4,43.858121,3,67.183560,4
3,5.988401,1,6.098320,1,10.741301,1
4,23.121533,2,6.583696,1,46.280888,3
